# Calculate Limits

## Simulate First

You can just run the whole notebook

### Imports / Helper Functions

In [1]:
import sys
import math
import warnings

import psycopg2
import wrds
import gzip

import seaborn as sns
import os
import quandl
import json
import zipfile
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import functools
import requests
import io

import urllib.request
from urllib.error import HTTPError
# from html_table_parser.parser import HTMLTableParser
import re

import plotnine as p9
from plotnine import ggplot, scale_x_date, guides, guide_legend, geom_bar, scale_y_continuous, \
    scale_color_identity, geom_line, geom_point, labs, theme_minimal, theme, element_blank, element_text, \
        geom_ribbon, geom_hline, aes, scale_size_manual, scale_color_manual, ggtitle

from datetime import datetime
import datetime

import pandas as pd
# import pandas_market_calendars as mcal
from pandas.plotting import autocorrelation_plot
import numpy as np
from numpy import cumsum, log, polyfit, sqrt, std, subtract
import scipy as sp
from scipy.stats import norm
import scipy.stats as stats

from statsmodels.tsa.stattools import coint
from statsmodels.graphics.tsaplots import plot_acf
import statsmodels.api as sm
from statsmodels.tsa.stattools import acf
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant

from collections import deque
from bisect import insort, bisect_left
from itertools import islice

pd.set_option('display.max_columns', None)

In [2]:
def max_drawdown(returns):
        local_max = [n for n in range(len(returns)-1) if ((n==0) and (returns[0] > returns[1])) or 
           ((n > 0) and  (returns[n-1]<returns[n]) and (returns[n+1]<returns[n]))] 
        
        local_min = [n for n in range(1,len(returns)) if ((n == len(returns)-1) and (returns[-1] < returns[-2])) or
                (returns[n-1]>returns[n]) and (returns[n+1]>returns[n])]
        
        def next_local_min(n):
            if [m for m in local_min if m > n]:
                return [m for m in local_min if m > n][0]
            else: return None
        
        drawdowns = [(n,next_local_min(n)) for n in local_max]
        drawdown_values = [returns[n] - returns[m] for (n,m) in drawdowns if m != None]
        if drawdown_values:
            return  np.max(drawdown_values)
        else: return 0.0

### Dividends

In [3]:
start_date = '2018-01-01'
end_date = '2023-02-28'
spy_divdata = pd.read_csv('spy_tickerdata.csv')[['date','dividend']].sort_values(by='date').reset_index(drop=True)
spy_divdata = spy_divdata.loc[(spy_divdata['date'] >= start_date) & (spy_divdata['date'] <= end_date)].copy().reset_index(drop=True)
spy_divdata['date'] = pd.to_datetime(spy_divdata['date'])
trading_days = spy_divdata['date']
spy_divdata = spy_divdata.loc[spy_divdata['dividend'] != 0]

# Step 1: Calculate the last day of the following month
spy_divdata['date'] = pd.to_datetime(spy_divdata['date'])  # Ensure 'date' is in datetime format
spy_divdata['end_of_next_month'] = spy_divdata['date'] + pd.offsets.MonthEnd(2)

# Step 2: Adjust to the nearest trading day

# This function finds the last trading day of the month or the nearest previous trading day
def find_pay_date(end_of_month, trading_days):
    if end_of_month in trading_days:
        return end_of_month
    else:
        # Find the nearest previous trading day
        eligible_days = trading_days[trading_days <= end_of_month]
        return eligible_days.max()  # The last trading day before or on the end_of_month

# Apply the function to each row to determine the 'pay_date'
spy_divdata['pay_date'] = spy_divdata['end_of_next_month'].apply(lambda date: find_pay_date(date, trading_days))

# Optionally, you can drop the intermediate 'end_of_next_month' column if it's no longer needed
spy_divdata.drop(columns=['end_of_next_month'], inplace=True)

spy_divdata

,date,dividend,pay_date
51,2018-03-16,1.096780,2018-04-30
114,2018-06-15,1.246000,2018-07-31
182,2018-09-21,1.323000,2018-10-31
245,2018-12-21,1.435400,2019-01-31
301,2019-03-15,1.233100,2019-04-30
369,2019-06-21,1.431600,2019-07-31
432,2019-09-20,1.383620,2019-10-31
496,2019-12-20,1.570000,2020-01-31
557,2020-03-20,1.405560,2020-04-30
620,2020-06-19,1.366200,2020-07-31


### Simulations

In [4]:
data = pd.read_csv('combinedata.csv')
options = pd.read_csv('option_df.csv')

data['exdate'] = pd.to_datetime(data['exdate'])
options['exdate'] = pd.to_datetime(options['exdate'])


data['exdate_str'] = data['exdate'].dt.strftime('%Y%m%d')  
data['strikeID'] = data['exdate_str'] + '_' + data['strike_price'].astype(str)  
data.drop(columns=['exdate_str'], inplace=True)  

options['exdate_str'] = options['exdate'].dt.strftime('%Y%m%d')  
options['strikeID'] = options['exdate_str'] + '_' + options['strike_price'].astype(str)  
options.drop(columns=['exdate_str'], inplace=True)  

options['date'] = pd.to_datetime(options['date'])
data['date'] = pd.to_datetime(data['date'])

In [5]:
def create_simulations(options_subset, data, dropna_greeks=False):
    simulations = {}

    for index, row in options_subset.iterrows():
        strikeID = row['exdate'].strftime('%Y%m%d') + '_' + str(row['strike_price'])
        mask = (data['strikeID'] == strikeID) & (data['date'] >= row['date']) & (data['date'] <= row['close_date'])
        temp_df = data[mask].sort_values(by=['date', 'cp_flag'])

        shared_cols = ['date', 'exdate', 'strike_price', 'close', 'strikeID'] # 'expiry_indicator',  'adj_open', 'adj_close', 'adj_volume',
        greeks_cols = ['impl_volatility', 'delta'] # , 'gamma', 'vega', 'theta'
        call_specific_cols = ['cp_flag', 'best_bid', 'best_offer'] + greeks_cols # , 'volume', 'open_interest'
        put_specific_cols = call_specific_cols

        calls = temp_df[temp_df['cp_flag'] == 'C'][shared_cols + call_specific_cols].rename(columns={col: col + '_c' for col in call_specific_cols})
        puts = temp_df[temp_df['cp_flag'] == 'P'][shared_cols + put_specific_cols].rename(columns={col: col + '_p' for col in put_specific_cols})

        merged_df = pd.merge(calls, puts, on=shared_cols, how='outer')

        if dropna_greeks:
            greeks_cols_c = [col + '_c' for col in greeks_cols]
            greeks_cols_p = [col + '_p' for col in greeks_cols]
            merged_df = merged_df.dropna(subset=greeks_cols_c + greeks_cols_p, how='any')

        merged_df['delta_sum'] = merged_df['delta_c'].fillna(0) + merged_df['delta_p'].fillna(0)
        merged_df['shares_held'] = -1 * merged_df['delta_sum']

        merged_df = merged_df.sort_values(by='date')
        merged_df['sharechange'] = merged_df['shares_held'].diff()

        simulations[row['date'].strftime('%Y-%m-%d')] = merged_df

    return simulations

In [60]:
%%time
simulations = create_simulations(options, data, dropna_greeks=True)

CPU times: total: 8min 34s
Wall time: 8min 34s


In [61]:
len(trading_days) - len(simulations) # Adds up to contracts with missing greeks

158

In [62]:
filtered_simulations = {}

for key, df in simulations.items():
    # Ensure 'date' column is in datetime64 dtype
    df['date'] = pd.to_datetime(df['date'])
    
    # Find the range of trading days for each simulation
    start_date = df['date'].min()
    end_date = df['date'].max()
    
    # Generate the expected range of trading days
    expected_trading_days = trading_days[(trading_days >= start_date) & (trading_days <= end_date)]
    
    # Check if all expected trading days are present in the DataFrame
    actual_trading_days = df['date'].unique()
    actual_trading_days = pd.to_datetime(actual_trading_days)  # Ensure this is in datetime format for comparison
    
    # Use .isin() to check if each item in expected_trading_days is in actual_trading_days, then check if all are True
    if expected_trading_days.isin(actual_trading_days).all():
        # If there are no gaps, add the DataFrame to the filtered_simulations dictionary
        filtered_simulations[key] = df

In [63]:
len(trading_days) - len(filtered_simulations)

325

In [64]:
for key, df in list(filtered_simulations.items())[0:1]: 
    print(f"DataFrame for {key}:")
    print(df.columns)
    display(df)  
    print("\n")

DataFrame for 2018-01-03:
Index(['date', 'exdate', 'strike_price', 'close', 'strikeID', 'cp_flag_c',
       'best_bid_c', 'best_offer_c', 'impl_volatility_c', 'delta_c',
       'cp_flag_p', 'best_bid_p', 'best_offer_p', 'impl_volatility_p',
       'delta_p', 'delta_sum', 'shares_held', 'sharechange'],
      dtype='object')


,date,exdate,strike_price,close,strikeID,cp_flag_c,best_bid_c,best_offer_c,impl_volatility_c,delta_c,cp_flag_p,best_bid_p,best_offer_p,impl_volatility_p,delta_p,delta_sum,shares_held,sharechange
0,2018-01-03,2018-02-02,270.0,270.47,20180202_270.0,C,2.65,2.69,0.072398,0.562218,P,1.77,1.80,0.069568,-0.442522,0.119696,-0.119696,NaN
1,2018-01-04,2018-02-02,270.0,271.61,20180202_270.0,C,3.45,3.49,0.077848,0.632690,P,1.43,1.46,0.074706,-0.366922,0.265768,-0.265768,-0.146072
2,2018-01-05,2018-02-02,270.0,273.42,20180202_270.0,C,4.80,4.84,0.083820,0.727658,P,1.02,1.04,0.081783,-0.270119,0.457539,-0.457539,-0.191771
3,2018-01-08,2018-02-02,270.0,273.92,20180202_270.0,C,5.17,5.20,0.089623,0.749240,P,0.81,0.82,0.082307,-0.234696,0.514544,-0.514544,-0.057005
4,2018-01-09,2018-02-02,270.0,274.54,20180202_270.0,C,5.67,5.72,0.093639,0.773171,P,0.83,0.84,0.091492,-0.223283,0.549888,-0.549888,-0.035344
5,2018-01-10,2018-02-02,270.0,274.12,20180202_270.0,C,5.27,5.33,0.092791,0.759070,P,0.85,0.87,0.089866,-0.235707,0.523363,-0.523363,0.026525
6,2018-01-11,2018-02-02,270.0,276.12,20180202_270.0,C,6.76,6.91,0.090857,0.854846,P,0.56,0.58,0.097319,-0.162272,0.692574,-0.692574,-0.169211
7,2018-01-12,2018-02-02,270.0,277.92,20180202_270.0,C,8.45,8.60,0.102281,0.890014,P,0.43,0.44,0.107499,-0.121856,0.768158,-0.768158,-0.075584
8,2018-01-16,2018-02-02,270.0,276.97,20180202_270.0,C,7.82,7.97,0.130638,0.827166,P,0.60,0.61,0.121851,-0.157003,0.670163,-0.670163,0.097995
9,2018-01-17,2018-02-02,270.0,279.61,20180202_270.0,C,10.03,10.25,0.131682,0.904279,P,0.38,0.39,0.135023,-0.101488,0.802791,-0.802791,-0.132628


### PnL and Misc for future calcs

In [75]:
def calculate_realized_PL(df, long_op=True):
    df = df.reset_index(drop=True)
    
    # Vectorized initial operations for stock
    df['stock_pos'] = np.where(long_op, df['shares_held'], -df['shares_held'])
    df = df.drop(columns=['shares_held'])
    df['pos_change'] = np.where(long_op, df['sharechange'], -df['sharechange'])
    df = df.drop(columns=['sharechange'])
    df.loc[0, 'pos_change'] = df.loc[0, 'stock_pos']
    
    df['change_cost_basis'] = df['pos_change'] * df['close']
    df['stock_cost_basis'] = df['change_cost_basis'].cumsum()
    df['daily_stock_value'] = df['stock_pos'] * df['close']
    df['stock_PL'] = df['daily_stock_value'] - df['stock_cost_basis']

    # Initial option value and vectorized daily option value calculation
    df['option_cost_basis'] = df.loc[0, 'best_offer_c'] + df.loc[0, 'best_offer_p'] if long_op else -df.loc[0, 'best_bid_c'] - df.loc[0, 'best_bid_p']
    df['change_cost_basis_op'] = 0.0
    df.loc[0, 'change_cost_basis_op'] = df.loc[0, 'option_cost_basis']
    df['daily_option_value'] = np.where(long_op, df['best_bid_c'] + df['best_bid_p'], -(df['best_offer_c'] + df['best_offer_p']))
    df['option_PL'] = df['daily_option_value']- df['option_cost_basis']

    # Column to track total positions, PL, and cash flow after positions are closed
    df['total_cost_basis'] = df['stock_cost_basis'] + df['option_cost_basis']
    df['total_pos_value'] = df['daily_stock_value'] + df['daily_option_value']
    df['total_PL'] = df['stock_PL'] + df['option_PL']
    df['realized_stock_PL'] = 0.0
    df['realized_option_PL'] = 0.0
    df['realized_PL'] = 0.0

    # Misc
    df['UID'] = df['strikeID'] + '_' + str(df.loc[0, 'date'].date())
    df['to_open'] = 0
    df.loc[0, 'to_open'] = 1
#    df['gross_trades_value'] = abs(df['to_open'] * df['option_cost_basis']) + abs(df['change_cost_basis']) # Need to do this at end

    # Close positions on final day
    final_row_index = len(df) - 1
    df.loc[final_row_index, 'realized_stock_PL'] = df.loc[final_row_index, 'stock_PL']
    df.loc[final_row_index, 'realized_option_PL'] = df.loc[final_row_index, 'option_PL']
    df.loc[final_row_index, 'realized_PL'] = df.loc[final_row_index, 'total_PL']
#    df.loc[final_row_index, 'gross_trades_value'] = abs(df.loc[final_row_index, 'daily_option_value']) + abs(df.loc[final_row_index - 1, 'stock_pos']) * df.loc[final_row_index, 'close'] 

    final_close_price = df.loc[final_row_index, 'close']
    df.loc[final_row_index, 'stock_pos'] = 0
    df.loc[final_row_index, 'pos_change'] = - df.loc[final_row_index - 1, 'pos_change'] if final_row_index > 0 else 0
    df.loc[final_row_index, 'change_cost_basis'] = df.loc[final_row_index, 'pos_change'] * final_close_price
    df.loc[final_row_index, 'stock_cost_basis'] = 0
    df.loc[final_row_index, 'daily_stock_value'] = 0
    df.loc[final_row_index, 'stock_PL'] = 0

    df.loc[final_row_index, 'option_cost_basis'] = 0
    df.loc[final_row_index, 'change_cost_basis_op'] = -df.loc[final_row_index, 'daily_option_value']
    df.loc[final_row_index, 'daily_option_value'] = 0
    df.loc[final_row_index, 'option_PL'] = 0

    df.loc[final_row_index, 'total_cost_basis'] = 0
    df.loc[final_row_index, 'total_pos_value'] = 0
    df.loc[final_row_index, 'total_PL'] = 0
    
    return df

*Cash - total cost basis + net realized PL* gives you how much cash is available to use

In [79]:
simulations_long = {date: calculate_realized_PL(df.copy(), long_op=True) for date, df in filtered_simulations.items()}
simulations_short = {date: calculate_realized_PL(df.copy(), long_op=False) for date, df in filtered_simulations.items()}

In [80]:
for key, df in list(simulations_short.items())[0:1]: 
    print(f"DataFrame for {key}:")
    print(df.columns)
    display(df)  
    print("\n")

DataFrame for 2018-01-03:
Index(['date', 'exdate', 'strike_price', 'close', 'strikeID', 'cp_flag_c',
       'best_bid_c', 'best_offer_c', 'impl_volatility_c', 'delta_c',
       'cp_flag_p', 'best_bid_p', 'best_offer_p', 'impl_volatility_p',
       'delta_p', 'delta_sum', 'stock_pos', 'pos_change', 'change_cost_basis',
       'stock_cost_basis', 'daily_stock_value', 'stock_PL',
       'option_cost_basis', 'change_cost_basis_op', 'daily_option_value',
       'option_PL', 'total_cost_basis', 'total_pos_value', 'total_PL',
       'realized_stock_PL', 'realized_option_PL', 'realized_PL', 'UID',
       'to_open'],
      dtype='object')


,date,exdate,strike_price,close,strikeID,cp_flag_c,best_bid_c,best_offer_c,impl_volatility_c,delta_c,cp_flag_p,best_bid_p,best_offer_p,impl_volatility_p,delta_p,delta_sum,stock_pos,pos_change,change_cost_basis,stock_cost_basis,daily_stock_value,stock_PL,option_cost_basis,change_cost_basis_op,daily_option_value,option_PL,total_cost_basis,total_pos_value,total_PL,realized_stock_PL,realized_option_PL,realized_PL,UID,to_open
0,2018-01-03,2018-02-02,270.0,270.47,20180202_270.0,C,2.65,2.69,0.072398,0.562218,P,1.77,1.80,0.069568,-0.442522,0.119696,0.119696,0.119696,32.374177,32.374177,32.374177,0.000000,-4.42,-4.42,-4.49,-0.07,27.954177,27.884177,-0.070000,0.000000,0.00,0.000000,20180202_270.0_2018-01-03,1
1,2018-01-04,2018-02-02,270.0,271.61,20180202_270.0,C,3.45,3.49,0.077848,0.632690,P,1.43,1.46,0.074706,-0.366922,0.265768,0.265768,0.146072,39.674616,72.048793,72.185246,0.136453,-4.42,0.00,-4.95,-0.53,67.628793,67.235246,-0.393547,0.000000,0.00,0.000000,20180202_270.0_2018-01-03,0
2,2018-01-05,2018-02-02,270.0,273.42,20180202_270.0,C,4.80,4.84,0.083820,0.727658,P,1.02,1.04,0.081783,-0.270119,0.457539,0.457539,0.191771,52.434027,124.482820,125.100313,0.617494,-4.42,0.00,-5.88,-1.46,120.062820,119.220313,-0.842506,0.000000,0.00,0.000000,20180202_270.0_2018-01-03,0
3,2018-01-08,2018-02-02,270.0,273.92,20180202_270.0,C,5.17,5.20,0.089623,0.749240,P,0.81,0.82,0.082307,-0.234696,0.514544,0.514544,0.057005,15.614810,140.097629,140.943892,0.846263,-4.42,0.00,-6.02,-1.60,135.677629,134.923892,-0.753737,0.000000,0.00,0.000000,20180202_270.0_2018-01-03,0
4,2018-01-09,2018-02-02,270.0,274.54,20180202_270.0,C,5.67,5.72,0.093639,0.773171,P,0.83,0.84,0.091492,-0.223283,0.549888,0.549888,0.035344,9.703342,149.800971,150.966252,1.165280,-4.42,0.00,-6.56,-2.14,145.380971,144.406252,-0.974720,0.000000,0.00,0.000000,20180202_270.0_2018-01-03,0
5,2018-01-10,2018-02-02,270.0,274.12,20180202_270.0,C,5.27,5.33,0.092791,0.759070,P,0.85,0.87,0.089866,-0.235707,0.523363,0.523363,-0.026525,-7.271033,142.529938,143.464266,0.934327,-4.42,0.00,-6.20,-1.78,138.109938,137.264266,-0.845673,0.000000,0.00,0.000000,20180202_270.0_2018-01-03,0
6,2018-01-11,2018-02-02,270.0,276.12,20180202_270.0,C,6.76,6.91,0.090857,0.854846,P,0.56,0.58,0.097319,-0.162272,0.692574,0.692574,0.169211,46.722541,189.252480,191.233533,1.981053,-4.42,0.00,-7.49,-3.07,184.832480,183.743533,-1.088947,0.000000,0.00,0.000000,20180202_270.0_2018-01-03,0
7,2018-01-12,2018-02-02,270.0,277.92,20180202_270.0,C,8.45,8.60,0.102281,0.890014,P,0.43,0.44,0.107499,-0.121856,0.768158,0.768158,0.075584,21.006305,210.258785,213.486471,3.227687,-4.42,0.00,-9.04,-4.62,205.838785,204.446471,-1.392313,0.000000,0.00,0.000000,20180202_270.0_2018-01-03,0
8,2018-01-16,2018-02-02,270.0,276.97,20180202_270.0,C,7.82,7.97,0.130638,0.827166,P,0.60,0.61,0.121851,-0.157003,0.670163,0.670163,-0.097995,-27.141675,183.117110,185.615046,2.497936,-4.42,0.00,-8.58,-4.16,178.697110,177.035046,-1.662064,0.000000,0.00,0.000000,20180202_270.0_2018-01-03,0
9,2018-01-17,2018-02-02,270.0,279.61,20180202_270.0,C,10.03,10.25,0.131682,0.904279,P,0.38,0.39,0.135023,-0.101488,0.802791,0.802791,0.132628,37.084115,220.201225,224.468392,4.267167,-4.42,0.00,-10.64,-6.22,215.781225,213.828392,-1.952833,0.000000,0.00,0.000000,20180202_270.0_2018-01-03,0


## Trading Strat / Positions / Limits

### Trading Strat

In [81]:
# Replace with actual model free IV calculations. Make sure to select for only the dates present in simulations
# (use loop below with df.loc[0, 'date']

MF_IV = pd.Series([0.120] * len(filtered_simulations))
MF_IV

0      0.12
1      0.12
2      0.12
3      0.12
4      0.12
       ... 
968    0.12
969    0.12
970    0.12
971    0.12
972    0.12
Length: 973, dtype: float64

In [82]:
temp_data = []

for key, df in filtered_simulations.items():
    temp_data.append({'date': key, 'BS_Call_IV': df.loc[0, 'impl_volatility_c']})

IV_compare = pd.DataFrame(temp_data)

IV_compare['MF_Call_IV'] = MF_IV.reset_index(drop=True)
IV_compare['IV_diff'] = IV_compare['MF_Call_IV'] - IV_compare['BS_Call_IV']

In [83]:
def trade_strategy_1(x):
    if x > 0.002:
        return 1
    elif x < -0.002:
        return -1
    else:
        return 0

def trade_strategy_2(x):
    if x > 0.005:
        return 1
    else:
        return 0

In [84]:
strat_dict = {'trade_1': trade_strategy_1, 'trade_2': trade_strategy_2}

In [85]:
for key, func in strat_dict.items():
    IV_compare[key] = IV_compare['IV_diff'].apply(func)
IV_compare

,date,BS_Call_IV,MF_Call_IV,IV_diff,trade_1,trade_2
0,2018-01-03,0.072398,0.12,0.047602,1,1
1,2018-01-04,0.071179,0.12,0.048821,1,1
2,2018-01-10,0.080641,0.12,0.039359,1,1
3,2018-01-11,0.076890,0.12,0.043110,1,1
4,2018-01-12,0.079350,0.12,0.040650,1,1
...,...,...,...,...,...,...
968,2023-02-22,0.202075,0.12,-0.082075,-1,0
969,2023-02-23,0.196720,0.12,-0.076720,-1,0
970,2023-02-24,0.201088,0.12,-0.081088,-1,0
971,2023-02-27,0.194675,0.12,-0.074675,-1,0


In [86]:
def generate_trades_dfs(strat_dict = strat_dict, initial_df = IV_compare):
    
    trades_dfs = {}
    
    for key in strat_dict.keys():

        dfs_to_combine = []
        
        for index, row in initial_df.iterrows():
            date = row['date']
            trade = row[key]
            iv_diff = row['IV_diff']
            
            if trade == 1 and date in simulations_long:
                df_to_add = simulations_long[date].copy()
            elif trade == -1 and date in simulations_short:
                df_to_add = simulations_short[date].copy()
            else:
                # Skip if 'trade' is 0 or the date is not in the dictionaries
                continue
            
            # Add 'trade' & 'IV_diff' column
            df_to_add['IV_diff'] = iv_diff # Needed for position calculation 
            df_to_add[key] = trade  # Include the 'trade' value
            dfs_to_combine.append(df_to_add)
        
        # Concatenate all collected DataFrames
        trades_dfs[key] = pd.concat(dfs_to_combine, ignore_index=True)
        trades_dfs[key] = trades_dfs[key].sort_values(by=['date', 'exdate', 'strike_price', 'to_open']).reset_index(drop=True)

    return trades_dfs

In [87]:
%%time
trades_dfs = generate_trades_dfs()

trades_dfs['trade_1']

CPU times: total: 609 ms
Wall time: 584 ms


,date,exdate,strike_price,close,strikeID,cp_flag_c,best_bid_c,best_offer_c,impl_volatility_c,delta_c,cp_flag_p,best_bid_p,best_offer_p,impl_volatility_p,delta_p,delta_sum,stock_pos,pos_change,change_cost_basis,stock_cost_basis,daily_stock_value,stock_PL,option_cost_basis,change_cost_basis_op,daily_option_value,option_PL,total_cost_basis,total_pos_value,total_PL,realized_stock_PL,realized_option_PL,realized_PL,UID,to_open,IV_diff,trade_1
0,2018-01-03,2018-02-02,270.0,270.47,20180202_270.0,C,2.65,2.69,0.072398,0.562218,P,1.77,1.80,0.069568,-0.442522,0.119696,-0.119696,-0.119696,-32.374177,-32.374177,-32.374177,0.000000,4.49,4.49,4.42,-0.07,-27.884177,-27.954177,-0.070000,0.000000,0.00,0.000000,20180202_270.0_2018-01-03,1,0.047602,1
1,2018-01-04,2018-02-02,270.0,271.61,20180202_270.0,C,3.45,3.49,0.077848,0.632690,P,1.43,1.46,0.074706,-0.366922,0.265768,-0.265768,-0.146072,-39.674616,-72.048793,-72.185246,-0.136453,4.49,0.00,4.88,0.39,-67.558793,-67.305246,0.253547,0.000000,0.00,0.000000,20180202_270.0_2018-01-03,0,0.047602,1
2,2018-01-04,2018-02-02,272.0,271.61,20180202_272.0,C,2.13,2.17,0.071179,0.500445,P,2.12,2.15,0.068201,-0.508652,-0.008207,0.008207,0.008207,2.229103,2.229103,2.229103,0.000000,4.32,4.32,4.25,-0.07,6.549103,6.479103,-0.070000,0.000000,0.00,0.000000,20180202_272.0_2018-01-04,1,0.048821,1
3,2018-01-05,2018-02-02,270.0,273.42,20180202_270.0,C,4.80,4.84,0.083820,0.727658,P,1.02,1.04,0.081783,-0.270119,0.457539,-0.457539,-0.191771,-52.434027,-124.482820,-125.100313,-0.617494,4.49,0.00,5.82,1.33,-119.992820,-119.280313,0.712506,0.000000,0.00,0.000000,20180202_270.0_2018-01-03,0,0.047602,1
4,2018-01-05,2018-02-02,272.0,273.42,20180202_272.0,C,3.27,3.32,0.076904,0.622425,P,1.48,1.52,0.074806,-0.379226,0.243199,-0.243199,-0.251406,-68.739429,-66.510325,-66.495471,0.014855,4.32,0.00,4.75,0.43,-62.190325,-61.745471,0.444855,0.000000,0.00,0.000000,20180202_272.0_2018-01-04,0,0.048821,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14541,2023-02-28,2023-03-24,399.0,396.26,20230324_399.0,C,7.03,7.05,0.207028,0.465189,P,8.85,8.87,0.170603,-0.573872,-0.108683,0.000000,-0.041990,-16.638957,0.000000,0.000000,0.000000,0.00,15.92,0.00,0.00,0.000000,0.000000,0.000000,-0.416764,3.79,3.373236,20230324_399.0_2023-02-21,0,-0.091530,-1
14542,2023-02-28,2023-03-24,399.0,396.26,20230324_399.0,C,7.03,7.05,0.207028,0.465189,P,8.85,8.87,0.170603,-0.573872,-0.108683,0.000000,-0.041990,-16.638957,0.000000,0.000000,0.000000,0.00,15.92,0.00,0.00,0.000000,0.000000,0.000000,-0.398548,2.83,2.431452,20230324_399.0_2023-02-22,0,-0.082075,-1
14543,2023-02-28,2023-03-24,401.0,396.26,20230324_401.0,C,5.99,6.01,0.202116,0.424099,P,9.85,9.87,0.164628,-0.623364,-0.199265,0.000000,-0.037696,-14.937417,0.000000,0.000000,0.000000,0.00,15.88,0.00,0.00,0.000000,0.000000,0.000000,-0.085759,1.74,1.654241,20230324_401.0_2023-02-23,0,-0.076720,-1
14544,2023-02-28,2023-03-31,396.0,396.26,20230331_396.0,C,9.55,9.58,0.207305,0.519384,P,8.15,8.17,0.174174,-0.498412,0.020972,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,17.75,0.00,0.00,0.000000,0.000000,0.000000,0.000000,-0.05,-0.050000,20230331_396.0_2023-02-28,1,-0.087305,-1


In [88]:
KAPITAL = 1e7

In [92]:
def pos_size(IV_diff, strike_price, option_cost_basis, UID, key):
    factor = .01* strike_price # Adjust this based on whatever

    if option_cost_basis == 0:
        # Get the first matching 'option_cost_basis' safely
        filtered_df = trades_dfs[key].loc[trades_dfs[key]['UID'] == UID, 'option_cost_basis']

        option_cost_basis = filtered_df.iloc[0] 

    posSize = round((abs(IV_diff) / factor) * KAPITAL / abs(option_cost_basis+.000000000001)) # some too close to zero, trigger infinity
    
    return posSize
    
    # kept it simple instead of my earlier complex one

In [90]:
for key, df in trades_dfs.items():
    trades_dfs[key] = df.drop(columns=[col for col in df.columns if col.endswith('_p') or col.endswith('_c')]).copy()

In [93]:
for key, df in trades_dfs.items():

    df['pos_size'] = df.apply(lambda row: pos_size(row['IV_diff'], row['strike_price'], row['option_cost_basis'], row['UID'], key), axis=1)
    lot_size = 100 * df['pos_size']
    
    for col in ['stock_pos',	'pos_change',	'change_cost_basis',	'stock_cost_basis',	'daily_stock_value',	'stock_PL',	'option_cost_basis',
                'change_cost_basis_op', 'daily_option_value',	'option_PL',	'total_cost_basis',	'total_pos_value',	'total_PL',	'realized_stock_PL',
                'realized_option_PL',	'realized_PL']:
        df['sized_' + col] = lot_size * df[col]

In [94]:
trades_dfs['trade_1'].head()

,date,exdate,strike_price,close,strikeID,delta_sum,stock_pos,pos_change,change_cost_basis,stock_cost_basis,daily_stock_value,stock_PL,option_cost_basis,change_cost_basis_op,daily_option_value,option_PL,total_cost_basis,total_pos_value,total_PL,realized_stock_PL,realized_option_PL,realized_PL,UID,to_open,IV_diff,trade_1,pos_size,sized_stock_pos,sized_pos_change,sized_change_cost_basis,sized_stock_cost_basis,sized_daily_stock_value,sized_stock_PL,sized_option_cost_basis,sized_change_cost_basis_op,sized_daily_option_value,sized_option_PL,sized_total_cost_basis,sized_total_pos_value,sized_total_PL,sized_realized_stock_PL,sized_realized_option_PL,sized_realized_PL
0,2018-01-03,2018-02-02,270.0,270.47,20180202_270.0,0.119696,-0.119696,-0.119696,-32.374177,-32.374177,-32.374177,0.000000,4.49,4.49,4.42,-0.07,-27.884177,-27.954177,-0.070000,0.0,0.0,0.0,20180202_270.0_2018-01-03,1,0.047602,1,39266,-4.699983e+05,-4.699983e+05,-1.271204e+08,-1.271204e+08,-1.271204e+08,0.000000e+00,17630434.0,17630434.0,17355572.0,-274862.0,-1.094900e+08,-1.097649e+08,-2.748620e+05,0.0,0.0,0.0
1,2018-01-04,2018-02-02,270.0,271.61,20180202_270.0,0.265768,-0.265768,-0.146072,-39.674616,-72.048793,-72.185246,-0.136453,4.49,0.00,4.88,0.39,-67.558793,-67.305246,0.253547,0.0,0.0,0.0,20180202_270.0_2018-01-03,0,0.047602,1,39266,-1.043565e+06,-5.735663e+05,-1.557863e+08,-2.829068e+08,-2.834426e+08,-5.357981e+05,17630434.0,0.0,19161808.0,1531374.0,-2.652764e+08,-2.642808e+08,9.955759e+05,0.0,0.0,0.0
2,2018-01-04,2018-02-02,272.0,271.61,20180202_272.0,-0.008207,0.008207,0.008207,2.229103,2.229103,2.229103,0.000000,4.32,4.32,4.25,-0.07,6.549103,6.479103,-0.070000,0.0,0.0,0.0,20180202_272.0_2018-01-04,1,0.048821,1,41548,3.409844e+04,3.409844e+04,9.261478e+06,9.261478e+06,9.261478e+06,0.000000e+00,17948736.0,17948736.0,17657900.0,-290836.0,2.721021e+07,2.691938e+07,-2.908360e+05,0.0,0.0,0.0
3,2018-01-05,2018-02-02,270.0,273.42,20180202_270.0,0.457539,-0.457539,-0.191771,-52.434027,-124.482820,-125.100313,-0.617494,4.49,0.00,5.82,1.33,-119.992820,-119.280313,0.712506,0.0,0.0,0.0,20180202_270.0_2018-01-03,0,0.047602,1,39266,-1.796573e+06,-7.530080e+05,-2.058874e+08,-4.887942e+08,-4.912189e+08,-2.424650e+06,17630434.0,0.0,22852812.0,5222378.0,-4.711638e+08,-4.683661e+08,2.797728e+06,0.0,0.0,0.0
4,2018-01-05,2018-02-02,272.0,273.42,20180202_272.0,0.243199,-0.243199,-0.251406,-68.739429,-66.510325,-66.495471,0.014855,4.32,0.00,4.75,0.43,-62.190325,-61.745471,0.444855,0.0,0.0,0.0,20180202_272.0_2018-01-04,0,0.048821,1,41548,-1.010443e+06,-1.044542e+06,-2.855986e+08,-2.763371e+08,-2.762754e+08,6.171818e+04,17948736.0,0.0,19735300.0,1786564.0,-2.583884e+08,-2.565401e+08,1.848282e+06,0.0,0.0,0.0


In [95]:
for key, df in trades_dfs.items():
    # Calculate the multiplication of 'identifier' and 'pos_size'
    df['signed_pos']  = - df[key] * df['pos_size']
    
trades_dfs['trade_1'].head()

,date,exdate,strike_price,close,strikeID,delta_sum,stock_pos,pos_change,change_cost_basis,stock_cost_basis,daily_stock_value,stock_PL,option_cost_basis,change_cost_basis_op,daily_option_value,option_PL,total_cost_basis,total_pos_value,total_PL,realized_stock_PL,realized_option_PL,realized_PL,UID,to_open,IV_diff,trade_1,pos_size,sized_stock_pos,sized_pos_change,sized_change_cost_basis,sized_stock_cost_basis,sized_daily_stock_value,sized_stock_PL,sized_option_cost_basis,sized_change_cost_basis_op,sized_daily_option_value,sized_option_PL,sized_total_cost_basis,sized_total_pos_value,sized_total_PL,sized_realized_stock_PL,sized_realized_option_PL,sized_realized_PL,signed_pos
0,2018-01-03,2018-02-02,270.0,270.47,20180202_270.0,0.119696,-0.119696,-0.119696,-32.374177,-32.374177,-32.374177,0.000000,4.49,4.49,4.42,-0.07,-27.884177,-27.954177,-0.070000,0.0,0.0,0.0,20180202_270.0_2018-01-03,1,0.047602,1,39266,-4.699983e+05,-4.699983e+05,-1.271204e+08,-1.271204e+08,-1.271204e+08,0.000000e+00,17630434.0,17630434.0,17355572.0,-274862.0,-1.094900e+08,-1.097649e+08,-2.748620e+05,0.0,0.0,0.0,-39266
1,2018-01-04,2018-02-02,270.0,271.61,20180202_270.0,0.265768,-0.265768,-0.146072,-39.674616,-72.048793,-72.185246,-0.136453,4.49,0.00,4.88,0.39,-67.558793,-67.305246,0.253547,0.0,0.0,0.0,20180202_270.0_2018-01-03,0,0.047602,1,39266,-1.043565e+06,-5.735663e+05,-1.557863e+08,-2.829068e+08,-2.834426e+08,-5.357981e+05,17630434.0,0.0,19161808.0,1531374.0,-2.652764e+08,-2.642808e+08,9.955759e+05,0.0,0.0,0.0,-39266
2,2018-01-04,2018-02-02,272.0,271.61,20180202_272.0,-0.008207,0.008207,0.008207,2.229103,2.229103,2.229103,0.000000,4.32,4.32,4.25,-0.07,6.549103,6.479103,-0.070000,0.0,0.0,0.0,20180202_272.0_2018-01-04,1,0.048821,1,41548,3.409844e+04,3.409844e+04,9.261478e+06,9.261478e+06,9.261478e+06,0.000000e+00,17948736.0,17948736.0,17657900.0,-290836.0,2.721021e+07,2.691938e+07,-2.908360e+05,0.0,0.0,0.0,-41548
3,2018-01-05,2018-02-02,270.0,273.42,20180202_270.0,0.457539,-0.457539,-0.191771,-52.434027,-124.482820,-125.100313,-0.617494,4.49,0.00,5.82,1.33,-119.992820,-119.280313,0.712506,0.0,0.0,0.0,20180202_270.0_2018-01-03,0,0.047602,1,39266,-1.796573e+06,-7.530080e+05,-2.058874e+08,-4.887942e+08,-4.912189e+08,-2.424650e+06,17630434.0,0.0,22852812.0,5222378.0,-4.711638e+08,-4.683661e+08,2.797728e+06,0.0,0.0,0.0,-39266
4,2018-01-05,2018-02-02,272.0,273.42,20180202_272.0,0.243199,-0.243199,-0.251406,-68.739429,-66.510325,-66.495471,0.014855,4.32,0.00,4.75,0.43,-62.190325,-61.745471,0.444855,0.0,0.0,0.0,20180202_272.0_2018-01-04,0,0.048821,1,41548,-1.010443e+06,-1.044542e+06,-2.855986e+08,-2.763371e+08,-2.762754e+08,6.171818e+04,17948736.0,0.0,19735300.0,1786564.0,-2.583884e+08,-2.565401e+08,1.848282e+06,0.0,0.0,0.0,-41548


### PL Positions

In [96]:
PL_temp_dfs = {}  
for key, df in trades_dfs.items():
    # Group by 'dates' and sum the columns of interest
    columns_to_sum = ['signed_pos'] 
    for col in ['stock_pos', 'change_cost_basis',	'stock_cost_basis',	'daily_stock_value',	'stock_PL',	'option_cost_basis', 'change_cost_basis_op',
                'daily_option_value',	'option_PL',	'total_cost_basis',	'total_pos_value',	'total_PL',	'realized_stock_PL',
                'realized_option_PL',	'realized_PL']:
        columns_to_sum.append('sized_' + col)
    
    
    grouped_df = df[['date'] + columns_to_sum].groupby('date').sum().reset_index()
    
    # Reindex the resulting DataFrame to match 'trading_days', filling missing values with 0 or other appropriate value
    pl_df = grouped_df.set_index('date').reindex(trading_days).fillna(0).reset_index()
    pl_df.rename(columns={'index': 'date'}, inplace=True)
    
    # Assign the processed DataFrame to PL_dfs with the same key
    PL_temp_dfs[key] = pl_df
    
PL_temp_dfs['trade_1'][70:85]

,date,signed_pos,sized_stock_pos,sized_change_cost_basis,sized_stock_cost_basis,sized_daily_stock_value,sized_stock_PL,sized_option_cost_basis,sized_change_cost_basis_op,sized_daily_option_value,sized_option_PL,sized_total_cost_basis,sized_total_pos_value,sized_total_PL,sized_realized_stock_PL,sized_realized_option_PL,sized_realized_PL
70,2018-04-13,322521.0,8.306508e+06,-3.564263e+08,2.243816e+09,2.202471e+09,-4.134573e+07,-316550117.0,9239900.0,-259742670.0,56807447.0,1.927266e+09,1.942728e+09,1.546171e+07,-1.360688e+07,11454128.0,-2.152757e+06
71,2018-04-16,305649.0,1.329785e+07,1.394053e+09,3.568583e+09,3.554915e+09,-1.366755e+07,-305684893.0,1550360.0,-262791624.0,42893269.0,3.262898e+09,3.292124e+09,2.922572e+07,-9.569996e+06,9314864.0,-2.551324e+05
72,2018-04-17,288289.0,1.641326e+07,1.146776e+09,4.410394e+09,4.434699e+09,2.430523e+07,-282144633.0,23787122.0,-286271993.0,-4127360.0,4.128249e+09,4.148427e+09,2.017787e+07,5.907704e+04,-246862.0,-1.877850e+05
73,2018-04-18,268008.0,1.489567e+07,3.710791e+07,4.000435e+09,4.027641e+09,2.720587e+07,-254805576.0,27313517.0,-257316010.0,-2510434.0,3.745629e+09,3.770325e+09,2.469544e+07,3.820114e+05,25540.0,4.075514e+05
74,2018-04-19,245966.0,1.161179e+07,-4.510521e+08,3.110873e+09,3.122294e+09,1.142129e+07,-233137221.0,14858280.0,-217171368.0,15965853.0,2.877736e+09,2.905123e+09,2.738714e+07,-6.558925e+06,6810075.0,2.511504e+05
75,2018-04-20,222071.0,8.244467e+06,-8.977619e+08,2.213111e+09,2.198057e+09,-1.505359e+07,-233137221.0,0.0,-193165484.0,39971737.0,1.979974e+09,2.004892e+09,2.491815e+07,0.000000e+00,0.0,0.000000e+00
76,2018-04-23,222071.0,6.259544e+06,1.249630e+08,1.687881e+09,1.668607e+09,-1.927468e+07,-198319767.0,34817454.0,-146375602.0,51944165.0,1.489561e+09,1.522231e+09,3.266949e+07,3.891312e+06,0.0,3.891312e+06
77,2018-04-24,193833.0,-7.885156e+05,-1.479783e+09,-1.741120e+08,-2.073638e+08,-3.325185e+07,-172228302.0,14926305.0,-121918725.0,50309577.0,-3.463403e+08,-3.292826e+08,1.705772e+07,-8.494587e+06,11165160.0,2.670573e+06
78,2018-04-25,182577.0,3.376849e+05,4.834718e+08,1.138987e+08,8.902388e+07,-2.487477e+07,-158679771.0,2413641.0,-118033784.0,40645987.0,-4.478112e+07,-2.900991e+07,1.577121e+07,-8.889616e+06,11134890.0,2.245274e+06
79,2018-04-26,168225.0,4.021101e+06,8.543350e+08,1.087181e+09,1.070859e+09,-1.632182e+07,-148144031.0,1210152.0,-113190470.0,34953561.0,9.390371e+08,9.576688e+08,1.863174e+07,-7.647958e+06,9325588.0,1.677630e+06


In [97]:
divvies = {}

for key, df in PL_temp_dfs.items():
    # Ensure 'date' in both DataFrames is in datetime format
    df['date'] = pd.to_datetime(df['date'])
    spy_divdata['date'] = pd.to_datetime(spy_divdata['date'])
    spy_divdata['pay_date'] = pd.to_datetime(spy_divdata['pay_date'])
    
    # First Merge: Merge to align 'pos_size' with 'spy_divdata' based on 'date'
    # Assuming 'pos_size' exists in your df
    temp_merged = pd.merge(spy_divdata, df[['date', 'signed_pos']], how='left', on='date')

    # Correctly multiply 'signed_pos' by 'dividend' column
    temp_merged['div'] = temp_merged['signed_pos'] * temp_merged['dividend']
    
    # Update the DataFrame in divvies with the final merged DataFrame
    divvies[key] = temp_merged

divvies['trade_1'].head(50)

,date,dividend,pay_date,signed_pos,div
0,2018-03-16,1.096780,2018-04-30,36078.0,39569.628840
1,2018-06-15,1.246000,2018-07-31,-75945.0,-94627.470000
2,2018-09-21,1.323000,2018-10-31,-153959.0,-203687.757000
3,2018-12-21,1.435400,2019-01-31,131807.0,189195.767800
4,2019-03-15,1.233100,2019-04-30,-33701.0,-41556.703100
5,2019-06-21,1.431600,2019-07-31,54925.0,78630.630000
6,2019-09-20,1.383620,2019-10-31,-6338.0,-8769.383560
7,2019-12-20,1.570000,2020-01-31,-68563.0,-107643.910000
8,2020-03-20,1.405560,2020-04-30,375538.0,527841.191280
9,2020-06-19,1.366200,2020-07-31,342295.0,467643.429000


In [98]:
for key, pl_df in PL_temp_dfs.items():
    
    # Get the corresponding DataFrame from divvies
    div_df = divvies[key]
    pl_df['date'] = pd.to_datetime(pl_df['date'])
    # Perform the left merge
    # Note: Ensure 'date' in PL_temp_dfs and 'pay_date' in divvies are of compatible types (preferably datetime)
    merged_df = pd.merge(pl_df, div_df[['pay_date', 'div']], how='left', left_on='date', right_on='pay_date')
    
    # Optionally, you can drop the 'pay_date' column after the merge if it's no longer needed
    merged_df.drop(columns=['pay_date'], inplace=True)
    
    # Assuming 'div' can be NaN, fill NaNs with 0 for accurate calculations
    merged_df['div'] = merged_df['div'].fillna(0)
    
    # Update 'sized_realized_stock_PL' and 'sized_realized_PL' by adding 'div'
    merged_df['sized_realized_stock_PL'] = merged_df['sized_realized_stock_PL'] + merged_df['div']
    merged_df['sized_realized_PL'] = merged_df['sized_realized_PL'] + merged_df['div']
    
    # Update the PL_temp_dfs dictionary with the merged and updated result
    PL_temp_dfs[key] = merged_df

In [99]:
PL_temp_dfs['trade_1'][70:85]

,date,signed_pos,sized_stock_pos,sized_change_cost_basis,sized_stock_cost_basis,sized_daily_stock_value,sized_stock_PL,sized_option_cost_basis,sized_change_cost_basis_op,sized_daily_option_value,sized_option_PL,sized_total_cost_basis,sized_total_pos_value,sized_total_PL,sized_realized_stock_PL,sized_realized_option_PL,sized_realized_PL,div
70,2018-04-13,322521.0,8.306508e+06,-3.564263e+08,2.243816e+09,2.202471e+09,-4.134573e+07,-316550117.0,9239900.0,-259742670.0,56807447.0,1.927266e+09,1.942728e+09,1.546171e+07,-1.360688e+07,11454128.0,-2.152757e+06,0.00000
71,2018-04-16,305649.0,1.329785e+07,1.394053e+09,3.568583e+09,3.554915e+09,-1.366755e+07,-305684893.0,1550360.0,-262791624.0,42893269.0,3.262898e+09,3.292124e+09,2.922572e+07,-9.569996e+06,9314864.0,-2.551324e+05,0.00000
72,2018-04-17,288289.0,1.641326e+07,1.146776e+09,4.410394e+09,4.434699e+09,2.430523e+07,-282144633.0,23787122.0,-286271993.0,-4127360.0,4.128249e+09,4.148427e+09,2.017787e+07,5.907704e+04,-246862.0,-1.877850e+05,0.00000
73,2018-04-18,268008.0,1.489567e+07,3.710791e+07,4.000435e+09,4.027641e+09,2.720587e+07,-254805576.0,27313517.0,-257316010.0,-2510434.0,3.745629e+09,3.770325e+09,2.469544e+07,3.820114e+05,25540.0,4.075514e+05,0.00000
74,2018-04-19,245966.0,1.161179e+07,-4.510521e+08,3.110873e+09,3.122294e+09,1.142129e+07,-233137221.0,14858280.0,-217171368.0,15965853.0,2.877736e+09,2.905123e+09,2.738714e+07,-6.558925e+06,6810075.0,2.511504e+05,0.00000
75,2018-04-20,222071.0,8.244467e+06,-8.977619e+08,2.213111e+09,2.198057e+09,-1.505359e+07,-233137221.0,0.0,-193165484.0,39971737.0,1.979974e+09,2.004892e+09,2.491815e+07,0.000000e+00,0.0,0.000000e+00,0.00000
76,2018-04-23,222071.0,6.259544e+06,1.249630e+08,1.687881e+09,1.668607e+09,-1.927468e+07,-198319767.0,34817454.0,-146375602.0,51944165.0,1.489561e+09,1.522231e+09,3.266949e+07,3.891312e+06,0.0,3.891312e+06,0.00000
77,2018-04-24,193833.0,-7.885156e+05,-1.479783e+09,-1.741120e+08,-2.073638e+08,-3.325185e+07,-172228302.0,14926305.0,-121918725.0,50309577.0,-3.463403e+08,-3.292826e+08,1.705772e+07,-8.494587e+06,11165160.0,2.670573e+06,0.00000
78,2018-04-25,182577.0,3.376849e+05,4.834718e+08,1.138987e+08,8.902388e+07,-2.487477e+07,-158679771.0,2413641.0,-118033784.0,40645987.0,-4.478112e+07,-2.900991e+07,1.577121e+07,-8.889616e+06,11134890.0,2.245274e+06,0.00000
79,2018-04-26,168225.0,4.021101e+06,8.543350e+08,1.087181e+09,1.070859e+09,-1.632182e+07,-148144031.0,1210152.0,-113190470.0,34953561.0,9.390371e+08,9.576688e+08,1.863174e+07,-7.647958e+06,9325588.0,1.677630e+06,0.00000


In [100]:
PL_dfs = {}  # New dictionary to store the processed DataFrames

for key, df in PL_temp_dfs.items():
    # Create a new DataFrame for PL_dfs
    pl_df = pd.DataFrame(index=df.index)
    
    # Assuming 'date' is the column you want to keep as is
    pl_df['date'] = df['date']
    
    # Sum 'sized_change_cost_basis' and 'sized_change_cost_basis_op'
    pl_df['gross_trades_value'] = abs(df['sized_change_cost_basis']) + abs(df['sized_change_cost_basis_op'])
    
    # Calculate cumulative sum of 'sized_realized_stock_PL'
    pl_df['stock_PL'] = df['sized_realized_stock_PL'].cumsum()
    pl_df['option_PL'] = df['sized_realized_option_PL'].cumsum()
    pl_df['net_PL'] = df['sized_realized_PL'].cumsum()

    # Positions & Cost Basis
    pl_df['cash'] = KAPITAL - df['sized_total_cost_basis'] + df['sized_total_cost_basis'] + pl_df['net_PL']
    pl_df['position_value'] = pl_df['cash'] + df['sized_total_pos_value']
    
    # Assign the processed DataFrame to the new dictionary
    PL_dfs[key] = pl_df

In [101]:
PL_dfs['trade_1'][60:85]

,date,gross_trades_value,stock_PL,option_PL,net_PL,cash,position_value
60,2018-03-29,6.681690e+08,-1.071707e+08,163178525.0,5.600785e+07,6.600785e+07,3.430335e+08
61,2018-04-02,1.178280e+09,-1.071707e+08,163178525.0,5.600785e+07,6.600785e+07,-8.519410e+08
62,2018-04-03,9.276178e+08,-1.071707e+08,163178525.0,5.600785e+07,6.600785e+07,2.607020e+07
63,2018-04-04,8.916095e+08,-1.071707e+08,163178525.0,5.600785e+07,6.600785e+07,8.654714e+08
64,2018-04-05,8.385115e+08,-1.071707e+08,163178525.0,5.600785e+07,6.600785e+07,1.676528e+09
65,2018-04-06,2.378697e+09,-1.071707e+08,163178525.0,5.600785e+07,6.600785e+07,-7.252374e+08
66,2018-04-09,6.707552e+08,-1.071707e+08,163178525.0,5.600785e+07,6.600785e+07,-1.099923e+08
67,2018-04-10,2.095662e+09,-1.071707e+08,163178525.0,5.600785e+07,6.600785e+07,1.945706e+09
68,2018-04-11,6.726273e+08,-1.071707e+08,163178525.0,5.600785e+07,6.600785e+07,1.288116e+09
69,2018-04-12,1.359193e+09,-1.249059e+08,174785445.0,4.987956e+07,5.987956e+07,2.570559e+09


In [ ]:
os.makedirs('simdata', exist_ok=True)

for strat, df in PL_dfs.items():
    csv_path = f'simdata/PL_{strat}.csv'
    df_long.to_csv(csv_path, index=False)

### Other things to consider (WIP)

In [104]:
# Aggregate same strike/expiration options to further reduce trading costs once positions have been determined, etc.
for key, df in trades_dfs.items():
    df = df.drop(columns=[col for col in df.columns if col.endswith('_p') or col.endswith('_c')])

aggregations = {
    'exdate': lambda x: x.iloc[0] if all(date == x.iloc[0] for date in x) else 'Variable',
    'strike_price': lambda x: x.iloc[0] if all(strike == x.iloc[0] for strike in x) else None,
    'close': lambda x: x.iloc[0] if all(strike == x.iloc[0] for strike in x) else None,
    'delta_sum':
    stock_pos
    pos_change
    change_cost_basis
    stock_cost_basis
    daily_stock_value
    stock_PL
    option_cost_basis
    daily_option_value
    option_PL
    total_cost_basis
    total_pos_value	total_PL
    realized_stock_PL
    realized_option_PL
    realized_PL
    UID
    to_open
    IV_diff	
}

SyntaxError: invalid syntax (1279976394.py, line 11)